In [1]:
%%capture --no-display
!curl -fsSL https://ollama.com/install.sh | sh

In [2]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)

In [3]:
%%capture --no-display
!ollama pull llama3.1 
!pip install ollama

In [5]:
import pandas as pd
import json
import requests

In [6]:
def get_response_from_server(prompt, model="llama3.1"):
    url = "http://127.0.0.1:11434/api/chat"  # Ollama API endpoint
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }
    try:
        response = requests.post(url, json=payload, stream=True)
        if response.status_code == 200:
            complete_response = ""
            for line in response.iter_lines():
                if line:  # Ignore empty lines
                    try:
                        chunk = json.loads(line)
                        if "message" in chunk and "content" in chunk["message"]:
                            complete_response += chunk["message"]["content"]
                    except (KeyError, json.JSONDecodeError) as e:
                        print("Warning: Skipped a line due to error -", str(e))
            return complete_response.strip()
        else:
            return f"Error: {response.status_code} - {response.text}"
    except requests.exceptions.RequestException as e:
        print("Error connecting to the server:", e)
        return None

**Zero Shot Prompt**

In [8]:
def predict_diabetes_single_zero_shot(data_row):
    prompt = f"""
You are a diabetes expert. Evaluate the following patient data and decide if the patient is diabetic.
Answer with only a single digit: 1 for diabetic, 0 for non-diabetic. No extra text.

Pregnancies: {data_row['Pregnancies']}
Glucose: {data_row['Glucose']}
BloodPressure: {data_row['BloodPressure']}
SkinThickness: {data_row['SkinThickness']}
Insulin: {data_row['Insulin']}
BMI: {data_row['BMI']}
DiabetesPedigreeFunction: {data_row['DiabetesPedigreeFunction']}
Age: {data_row['Age']}

Answer:"""
    return get_response_from_server(prompt)

def main():
    csv_file = '/kaggle/input/diabetes/diabetes_input.csv'
    df = pd.read_csv(csv_file)
    
    predictions = []
    for _, row in df.iterrows():
        prediction = predict_diabetes_single_zero_shot(row)
        predictions.append(prediction)
    
    df["Predicted_Outcome"] = predictions
    output_csv_file = "/kaggle/working/diabetes_predictions_llama3.1_zero_shot.csv"
    df.to_csv(output_csv_file, index=False)
    
    print(f"Predictions saved to {output_csv_file}")

if __name__ == "__main__":
    main()


Predictions saved to /kaggle/working/diabetes_predictions_llama3.1_zero_shot.csv


**One Shot Prompt**

In [9]:
def predict_diabetes_single_one_shot(data_row):
    prompt = f"""
You are a diabetes expert. Based on the patient data, decide if the patient is diabetic.
Answer with a single digit: 1 for diabetic, 0 for non-diabetic. No extra text.

Example:
Pregnancies: 1
Glucose: 85
BloodPressure: 66
SkinThickness: 29
Insulin: 0
BMI: 26.6
DiabetesPedigreeFunction: 0.351
Age: 31
Answer: 0

Now, evaluate this patient:
Pregnancies: {data_row['Pregnancies']}
Glucose: {data_row['Glucose']}
BloodPressure: {data_row['BloodPressure']}
SkinThickness: {data_row['SkinThickness']}
Insulin: {data_row['Insulin']}
BMI: {data_row['BMI']}
DiabetesPedigreeFunction: {data_row['DiabetesPedigreeFunction']}
Age: {data_row['Age']}

Answer:"""
    return get_response_from_server(prompt)

def main():
    csv_file = '/kaggle/input/diabetes/diabetes_input.csv'
    df = pd.read_csv(csv_file)
    
    predictions = []
    for _, row in df.iterrows():
        prediction = predict_diabetes_single_one_shot(row)
        predictions.append(prediction)
    
    df["Predicted_Outcome"] = predictions
    output_csv_file = "/kaggle/working/diabetes_predictions_llama3.1_one_shot.csv"
    df.to_csv(output_csv_file, index=False)
    
    print(f"Predictions saved to {output_csv_file}")

if __name__ == "__main__":
    main()

Predictions saved to /kaggle/working/diabetes_predictions_llama3.1_one_shot.csv


**Three Shot Prompt**

In [10]:
def predict_diabetes_single_three_shot(data_row):
    prompt = f"""
You are a diabetes expert. Use the following examples to decide if a patient is diabetic.
Answer with only a single digit: 1 for diabetic, 0 for non-diabetic. No extra text.

Example 1:
Pregnancies: 1
Glucose: 85
BloodPressure: 66
SkinThickness: 29
Insulin: 0
BMI: 26.6
DiabetesPedigreeFunction: 0.351
Age: 31
Answer: 0

Example 2:
Pregnancies: 6
Glucose: 148
BloodPressure: 72
SkinThickness: 35
Insulin: 0
BMI: 33.6
DiabetesPedigreeFunction: 0.627
Age: 50
Answer: 1

Example 3:
Pregnancies: 8
Glucose: 183
BloodPressure: 64
SkinThickness: 0
Insulin: 0
BMI: 23.3
DiabetesPedigreeFunction: 0.672
Age: 32
Answer: 1

Now, evaluate this patient:
Pregnancies: {data_row['Pregnancies']}
Glucose: {data_row['Glucose']}
BloodPressure: {data_row['BloodPressure']}
SkinThickness: {data_row['SkinThickness']}
Insulin: {data_row['Insulin']}
BMI: {data_row['BMI']}
DiabetesPedigreeFunction: {data_row['DiabetesPedigreeFunction']}
Age: {data_row['Age']}

Answer:"""
    return get_response_from_server(prompt)

def main():
    csv_file = '/kaggle/input/diabetes/diabetes_input.csv'
    df = pd.read_csv(csv_file)
    
    predictions = []
    for _, row in df.iterrows():
        prediction = predict_diabetes_single_three_shot(row)
        predictions.append(prediction)
    
    df["Predicted_Outcome"] = predictions
    output_csv_file = "/kaggle/working/diabetes_predictions_mistral_three_shot.csv"
    df.to_csv(output_csv_file, index=False)
    
    print(f"Predictions saved to {output_csv_file}")

if __name__ == "__main__":
    main()

Predictions saved to /kaggle/working/diabetes_predictions_mistral_three_shot.csv
